In [ ]:
import re, string, unicodedata
from bs4 import BeautifulSoup
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from tqdm import tqdm
tqdm.pandas()
from multiprocessing import Pool, cpu_count
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
cachedStopWords = stopwords.words("english")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
#define helper functions for text prep/cleanup

REPLACE_BY_SPACE_RE = re.compile(r'[(){}\/[\]:\&|@,;-]')
BAD_SYMBOLS_RE = re.compile('[^a-z ]')

ls = LancasterStemmer()
lemmatizer = WordNetLemmatizer()

In [ ]:
def regex(text):
  text = re.sub('#', 'number', text)
  text = re.sub('%', 'percent', text)
  text = re.sub('w/', 'with', text)
  text = re.sub('w/o', 'without', text)
  return text

In [ ]:
def replace_invalid_chars(text):
  text = re.sub(REPLACE_BY_SPACE_RE, ' ', text)
  return text

In [ ]:
def replace_bad_symbold(text):
  text = re.sub(BAD_SYMBOLS_RE, '', text)
  return text

In [ ]:
def remove_punctuation(text):
  text = re.sub(r'[^\w\s]', '', text)
  return text

In [ ]:
def remove_stopwords(text):
  return ' '.join([word for word in text.split() if word not in cachedStopWords])

In [ ]:
def nltk_process_text(text):
  processed_text = ''
  text = remove_stopwords(text)
  for word in word_tokenize(text):
    #if word in stopwords.words('english'):
    #  continue
    modified_word = lemmatizer.lemmatize(word)

    if modified_word == '.':
      #don't add space if period
      processed_text += modified_word
    elif processed_text != '':
      processed_text += ' ' + modified_word.casefold()
    else:
      processed_text = modified_word.casefold()
  return processed_text

In [ ]:
def stemming (text):
  processed_text = ''

  for word in word_tokenize(text):
    modified_word = ls.stem (word)

    if modified_word == '.':
      #don't add space if period
      processed_text += modified_word
    elif processed_text != '':
      processed_text += ' ' + modified_word.casefold()
    else:
      processed_text = modified_word.casefold()
  return processed_text


In [ ]:
def strip_html(text):
  soup = BeautifulSoup(text, "html_parser")
  return soup.get_text()

In [ ]:
def remove_between_square_brackets(text):
  return re.sub('\[[^}}*\]', '', text)

In [ ]:
def remove_blanks(text):
  processed_text = ''
  for word in word_tokenize(text):
    if word != '':
      if processed_text != '':
        #don't add extra space before a period
        if word == '.':
          processed_text += word
        else:
          processed_text += ' ' + word
      else:
        processed_text = word
  return processed_text

In [ ]:
def text_prep(text, params=[nltk_process_text, regex, remove_blanks], display_progress=True):
  if (display_progress):
    for param in params:
      print("applying:", param.__name__)
      text = text.progress_apply(param)
  else:
    for param in params:
      text = text.apply(param)

  return text

In [ ]:
#sample usage
#import pandas as pd
#text_str = "This text needs to be processed. Please do it for me."
#sample_text = [text_str]
#sample_df = pd.DataFrame(data = sample_text, columns = ['text'])

#preprocessed_text = text_prep(sample_df['text'])

#print("Original text:", text_str)
#print("Preprocessed text:", preprocessed_text[0])